In [1]:
import numpy as np
import torch.nn as nn
from dataset import CityscapesDataset
from torchvision.datasets import Cityscapes
import torch.nn.functional as F
from GenISP import *
from unet import *
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import os 
from tqdm import trange
from torchmetrics.detection.mean_ap import MeanAveragePrecision

In [2]:

# ALL CREDITS TO https://github.com/goldbattle/pytorch_unet/blob/master/script_train.py FOR DATASET CLASS
# path = "/Users/eleni/Cityscapes/"
path = R"C:\Users\eleni\Cityscapes"
# path = open(r"C:\Users\eleni\Downloads\leftImg8bit_trainvaltest\cityscapes"
img_data = CityscapesDataset(path, split='train', mode='fine')
test_data = CityscapesDataset(path, split='test', mode='fine')
batch_size = 8
img_batch = torch.utils.data.DataLoader(img_data, batch_size=batch_size, shuffle=True, num_workers=16)

c:\Users\eleni\Computer_Vision\Computer-Vision-Project\.venv\lib\site-packages\torch\utils\data\dataloader.py:560: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [3]:
# Hyperparameters and stuff
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.backends.mps.is_available():
    device = torch.device("mps")
print(device)

# Set model parameters
epochs = 10
learning_rate = 0.0001
in_c = 3

# Create the model
in_channels = 3 # Raw RGB image
hidden_channels = [16, 32, 128, 16, 64, 3]
out_features = 3 # GenISP output channels
PreNet = GenISPV2( in_channels, hidden_channels, out_features)
UNet = UNet(in_c, img_data.num_classes, 64)
# Net = nn.Sequential(PreNet,UNet)
torch.no_grad()
PreNet.to(device)
UNet.to(device)
train_loss = []
UNet.load_state_dict(torch.load(R"C:\Users\eleni\Computer_Vision\Computer-Vision-Project\model.pth", map_location=torch.device('cpu')))
PreNet.load_state_dict(torch.load(R"C:\Users\eleni\Computer_Vision\Computer-Vision-Project\prenet_model.pth", map_location=torch.device('cpu')))
# UNet.load_state_dict(torch.load("model.pth"),map_location=torch.device('cpu'))
# PreNet.load_state_dict(torch.load("prenet_model.pth"))

# Since our goal here is to segment, we will use the cross entropy loss function and 4 classes (background, road, sky, car)
criterion = nn.CrossEntropyLoss() # for segment loss. could also be reconstruction loss, then it is N1 loss 
num_classes = img_data.num_classes
print(num_classes)
optimizer = torch.optim.Adam(PreNet.parameters(), lr=learning_rate)

cpu
4


In [8]:
# running_accuracy = 0 
# total = 0 
# AP_50_loss = 0
# AP_75_loss = 0
# AP_loss = 0
# -------------------------------
# pixel_acc = 0.0
# dice=0.0
# precision = 0.0
# specificity = 0.0
# recall = 0.0
# --------------------------------
# dice_coeff = 0.0
# num_samples = 0
# --------------------------------
history_accuracy = []
testing_loss_metric = MeanAveragePrecision()
with torch.no_grad(): 
    # for data in test_data: 
    for idx_batch, (imagergb, labelmask, labelrgb) in enumerate(img_batch):
        # inputs, outputs = data 
        # outputs = outputs.to(torch.float32)
        imagergb = imagergb.to(device)
        labelmask = labelmask.to(device)
        labelrgb = labelrgb.to(device)
        
        output1 = PreNet(imagergb)
        output2 = F.pad(input=output1, pad=(2, 2, 2, 2))
        predicted_outputs = UNet(output2)
        pred_class = torch.zeros((predicted_outputs.size()[0], predicted_outputs.size()[2], predicted_outputs.size()[3]))
        for idx in range(0, predicted_outputs.size()[0]):
            pred_class[idx] = torch.argmax(predicted_outputs[idx], dim=0).cpu().int()

        pred_class = pred_class.unsqueeze(1).float()
        labelmask = labelmask.unsqueeze(1).float()

        acc_sum = (pred_class == labelmask).sum()
        acc = float(acc_sum) / (labelmask.size()[0]*labelmask.size()[2]*labelmask.size()[3])
        history_accuracy.append(acc)
# ----------------------------------------
#         predicted_outputs.resize_(8, 3, 128, 256)
#         predicted = torch.argmax(predicted_outputs, dim=1)
#         print(predicted.size())
#         intersection = torch.logical_and(predicted, labelrgb).sum().item()
#         union = predicted.sum().item() + labelrgb.sum().item()

#         dice_coeff += (2 * intersection) / (union + 1e-8)
#         num_samples += 1
#         print(dice_coeff)
        

# average_dice_coeff = dice_coeff / num_samples
print(history_accuracy)
print('Average accuracy:%.4f' % (sum(history_accuracy)/len(history_accuracy))) 

# -------------------------------------------------
        # predicted_outputs.resize_(8, 3, 128, 256)
        # p, d, pr, sp, r = calculate_overlap_metrics(labelrgb,predicted_outputs)
        # pixel_acc += p
        # dice += d
        # precision += pr
        # specificity += sp
        # recall += r
        # print(calculate_overlap_metrics(labelrgb,predicted_outputs))
        # -------------------------------------
        # testing_loss_metric.update(predicted_outputs, labelrgb)
        # loss = testing_loss_metric.compute()
        # AP_50_loss += loss["map_50"]
        # AP_75_loss += loss["map_75"]
        # AP_loss += loss["map"]
        # -----------------------------------------
        # # print('eleni')
        # _, predicted = torch.max(predicted_outputs, 1) 
        # total += labelrgb.size(0) 
        # running_accuracy += (predicted == labelrgb).sum().item() 
                
# print(average_dice_coeff)
    # print(pixel_acc)
    # print(dice)
    # print(precision)
    # print(specificity)
    # print(recall)    

c:\Users\eleni\Computer_Vision\Computer-Vision-Project\.venv\lib\site-packages\torch\utils\data\dataloader.py:560: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


[0.9528121948242188, 0.9704017639160156, 0.973480224609375, 0.9681129455566406, 0.95556640625, 0.9563674926757812, 0.9689674377441406, 0.9705238342285156, 0.9660148620605469, 0.9724159240722656, 0.9712562561035156, 0.964996337890625, 0.9597587585449219, 0.9572601318359375, 0.9643516540527344, 0.9689559936523438, 0.9746017456054688, 0.9644432067871094, 0.9555015563964844, 0.9666938781738281, 0.975067138671875, 0.9609718322753906, 0.9621124267578125, 0.9698066711425781, 0.9710922241210938, 0.9631233215332031, 0.9736366271972656, 0.9690322875976562, 0.9574432373046875, 0.9607505798339844, 0.9626998901367188, 0.9643516540527344, 0.966156005859375, 0.9637641906738281, 0.9666481018066406, 0.9750862121582031, 0.9529533386230469, 0.9664802551269531, 0.9672203063964844, 0.9649734497070312, 0.9667282104492188, 0.9659423828125, 0.9612808227539062, 0.9511337280273438, 0.9681472778320312, 0.9740104675292969, 0.9674186706542969, 0.9678497314453125, 0.9592094421386719, 0.9671134948730469, 0.952064514